In [2]:
import pandas as pd, boto3, json, re
from io import BytesIO
from datetime import datetime

read csv from s3 object


In [ ]:
with open("/home/jh97/MyWorks/Documents/.aws_cdt.json", "r") as file:
    content = json.load(file)
    key_id = content["access_key"]
    secret_key = content["secrect_access_key"]

client = boto3.client(
    "s3",
    region_name="us-east-1",
    aws_access_key_id=key_id,
    aws_secret_access_key=secret_key,
)

# panigator = client.get_paginator("list_objects_v2")
# result = panigator.paginate(Bucket="crawling-to-datalake", Prefix="scraped/")

# for i in result.build_full_result()["Contents"]:
#     if datetime.fromisoformat(
#         i["Key"].split("_")[-1].split(".")[0]
#     ) < datetime.now() and i["Key"].split("_")[0].endswith("tgdd"):
#         print(i["Key"])
#         break

resp = client.get_object(
    Bucket="crawling-to-datalake", Key="scraped/tgdd_products_2025-08-26.csv"
)

df = pd.read_csv(BytesIO(resp["Body"].read()))
df

concat 2 dataframes


In [61]:
df = pd.concat(
    [
        pd.read_csv("../data/scraped/tgdd_products_2025-08-29.csv"),
        pd.read_csv("../data/scraped/cellphones_products_2025-08-29.csv"),
    ],
    ignore_index=True,
)
df

,id,name,onsale_price,brand,category,os,cpu/chipset,gpu,ram,storage,rating,reviews_count,is_new,url,retailer,created_at,updated_at
0,328815,Laptop HP Envy X360 14 fa0050AU - A29H8PA (R5 ...,25890000.0,HP,Laptop,Windows 11 Home SL,AMD Ryzen 5 - 8640HS,Card tích hợp - AMD Radeon Graphics,16 GB LPDDR5 (Onboard) 6400 MHz,512 GB SSD NVMe PCIe Gen 4.0,5.0,280.0,True,https://www.thegioididong.com/laptop/hp-envy-x...,TGDD,2024,2025-08-29 17:08:46
1,334997,Laptop Acer Aspire Go AG15 31P 30M4 - NX.KRPSV...,9690000.0,Acer,Laptop,Windows 11 Home SL,Intel Core i3 Alder Lake chuỗi N - N305,Intel UHD Graphics,8 GB LPDDR5 (Onboard) 4800 MHz,"256 GB SSD NVMe PCIe (Có thể tháo ra, lắp than...",4.9,1674.0,True,https://www.thegioididong.com/laptop/acer-aspi...,TGDD,2024,2025-08-29 17:08:46
2,335227,Điện thoại vivo Y04 4GB/64GB,2940000.0,vivo,Smartphone,Android 14,Unisoc T7225 8 nhân,NaN,4 GB,64 GB,5.0,1408.0,True,https://www.thegioididong.com/dtdd/vivo-y04-64gb,TGDD,04/2025,2025-08-29 17:08:46
3,308697,Laptop MSI Gaming GF63 Thin 12VE - 460VN (i5 1...,19290000.0,MSI,Laptop,Windows 11 Home SL,Intel Core i5 Alder Lake - 12450H,"Card rời - NVIDIA GeForce RTX 4050, 6 GB",8 GB DDR4 (1 khe 8 GB + 1 khe rời) 3200 MHz,512 GB SSD NVMe PCIe Gen 4.0,4.9,1009.0,True,https://www.thegioididong.com/laptop/msi-gamin...,TGDD,2023,2025-08-29 17:08:46
4,333479,Điện thoại HONOR X9c 5G 12GB/256GB,8690000.0,HONOR,Smartphone,Android 14,Snapdragon 6 Gen 1 5G 8 nhân,NaN,12 GB,256 GB,4.9,3047.0,True,https://www.thegioididong.com/dtdd/honor-x9c,TGDD,01/2025,2025-08-29 17:08:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,54449,OPPO Reno9 Pro,NaN,OPPO,Smartphone,NaN,Snapdragon 888+,NaN,12 GB,128 GB,NaN,NaN,True,https://cellphones.com.vn/oppo-reno-9-pro.html,CELLPHONES,2022-10-04 08:12:47,2025-08-29 17:08:46
2542,54287,Honor X6,NaN,Hãng khác,Smartphone,Android 12,Mediatek,NaN,4 GB,128 GB,NaN,NaN,True,https://cellphones.com.vn/honor-x6.html,CELLPHONES,2022-09-29 14:45:22,2025-08-29 17:08:46
2543,77575,Xiaomi Redmi Note 13 Pro Plus 4G,NaN,Xiaomi,Smartphone,Android 13,Mediatek Dimensity 7200 Ultra (4 nm),NaN,8 GB,256 GB,NaN,NaN,True,https://cellphones.com.vn/xiaomi-redmi-note-13...,CELLPHONES,2024-02-26 15:05:43,2025-08-29 17:08:46
2544,100815,iPhone 16e 256GB | Chính hãng VN/A,17990000.0,Apple,Phone,iOS 18,Chip A18,NaN,NaN,256 GB,5.0,1.0,True,https://cellphones.com.vn/iphone-16e-256gb.html,CELLPHONES,2025-02-20 00:12:23,2025-08-29 17:08:46


cleaning


In [68]:
# change dtypes
df = df.astype({"onsale_price": "Int64", "reviews_count": "Int64"})

# remove null price products
df.drop(index=df[df["onsale_price"].isnull()].index)
df[df["onsale_price"].isnull()]

,id,name,onsale_price,brand,category,os,cpu/chipset,gpu,ram,storage,rating,reviews_count,is_new,url,retailer,created_at,updated_at
1141,89507,Mobell F209 4G,<NA>,Hãng khác,Phone,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,True,https://cellphones.com.vn/mobell-f209-4g.html,CELLPHONES,2024-09-04 14:39:27,2025-08-29 17:08:46
1142,55930,Redmi K60 Pro 5G,<NA>,Xiaomi,Smartphone,Android 13,Snapdragon 8 Gen 2 (4 nm),NaN,8 GB,128 GB,4.8,4,True,https://cellphones.com.vn/xiaomi-redmi-k60-pro...,CELLPHONES,2022-11-10 10:50:55,2025-08-29 17:08:46
1145,84346,vivo S19,<NA>,Vivo,Smartphone,Android 14,Snapdragon 7 Gen 3,NaN,8 GB,256 GB,NaN,<NA>,True,https://cellphones.com.vn/dien-thoai-vivo-s19....,CELLPHONES,2024-06-07 21:19:44,2025-08-29 17:08:46
1146,59296,Huawei P60 Pro,<NA>,Huawei,Smartphone,HarmonyOS 3.0,Snapdragon 8+ Gen 1 4G,NaN,8 GB,256 GB,5.0,1,True,https://cellphones.com.vn/huawei-p60-pro.html,CELLPHONES,2023-02-01 16:12:29,2025-08-29 17:08:46
1149,41491,Xiaomi 12 Lite,<NA>,Xiaomi,Smartphone,"Android 12, MIUI 13",Snapdragon 778G 5G 6 nm,NaN,6 GB,128 GB,5.0,1,True,https://cellphones.com.vn/xiaomi-12-lite.html,CELLPHONES,2021-12-14 08:57:20,2025-08-29 17:08:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,72216,Infinix Zero 30,<NA>,Hãng khác,Smartphone,Android 13,Mediatek Dimensity 8020 (6 nm),NaN,8 GB,256 GB,NaN,<NA>,True,https://cellphones.com.vn/infinix-zero-30.html,CELLPHONES,2023-11-09 14:21:36,2025-08-29 17:08:46
2540,89503,Mobell M139 4G,<NA>,Hãng khác,Phone,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,True,https://cellphones.com.vn/mobell-m139-4g.html,CELLPHONES,2024-09-04 14:26:49,2025-08-29 17:08:46
2541,54449,OPPO Reno9 Pro,<NA>,OPPO,Smartphone,NaN,Snapdragon 888+,NaN,12 GB,128 GB,NaN,<NA>,True,https://cellphones.com.vn/oppo-reno-9-pro.html,CELLPHONES,2022-10-04 08:12:47,2025-08-29 17:08:46
2542,54287,Honor X6,<NA>,Hãng khác,Smartphone,Android 12,Mediatek,NaN,4 GB,128 GB,NaN,<NA>,True,https://cellphones.com.vn/honor-x6.html,CELLPHONES,2022-09-29 14:45:22,2025-08-29 17:08:46


In [ ]:
# normalize brand
df["brand"] = (
    df["brand"]
    .str.upper()
    .replace(r"MACBOOK|IPHONE \(APPLE\)", "APPLE", regex=True)
    .replace(r"HÃNG KHÁC", "N/A", regex=True)
    .fillna("N/A")
)

# normalize os
# df["os"][df["retailer"] == "TGDD"].replace(r"(\s+\+|\().*", "", regex=True).sort_values().unique()

<IntegerArray>
[        0,    325000,    390000,    410000,    450000,    470000,    490000,
    500000,    530000,    540000,
 ...
  97990000, 105990000, 112990000, 114590000, 115990000, 171790000, 176690000,
 182490000, 194990000,      <NA>]
Length: 533, dtype: Int64